# Open-source cardiac MR Fingerprinting
This notebook provides the image reconstruction and parameter estimation methods required to reproduce the multi-scanner comparison carried out in the paper.

## Overview
In this notebook the cardiac MR Fingerprinting (cMRF) data acquired at four different scanners and the corresponding spin-echo reference sequences are reconstructed and $T_1$ and $T_2$ maps are estimated. Average $T_1$ and $T_2$ are calculated in circular ROIs for different tissue types represented in the phantom. 

This notebook utilises MRpro for reconstruction and parameter estimation: https://github.com/PTB-MR/mrpro and it is designed to be run via Google colab: 

<a target="_blank" href="https://colab.research.google.com/github/PTB-MR/cMRF/open_source_cmrf_scanner_comparison.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

It will take roughly 1h to run the notebook for all four datasets.

## Outline

1) Installation of MRpro

2) Download data from zenodo

3) Define image reconstruction and parameter estimation methods for cMRF and reference sequences

4) Define evaluation methods

5) Run through all datasets and calculate $T_1$ and $T_2$ maps

6) Visualise and evaluate results

### 1) Installation of MRpro
Install MRpro (currently a working github-branch but will be changed to pypi package as soon as possible) and import

In [ ]:
!pip install git+https://github.com/PTB-MR/mrpro.git@epg#egg=mrpro[notebook]

In [ ]:
# Imports
import numpy as np
import torch
import matplotlib.pyplot as plt
import tempfile
import zipfile

import zenodo_get
from matplotlib.colors import ListedColormap
from pathlib import Path
from einops import rearrange
from mrpro.utils import split_idx
from mrpro.data import KData, DcfData, IData, CsmData 
from mrpro.data.traj_calculators import KTrajectoryIsmrmrd
from mrpro.algorithms.reconstruction import DirectReconstruction
from mrpro.operators.models.EPG import EpgMrfFispWithPreparation
from mrpro.data.traj_calculators import KTrajectoryCartesian
from mrpro.operators.models import MonoExponentialDecay, InversionRecovery
from mrpro.operators import MagnitudeOp
    

### 2) Download data from zenodo

In [ ]:
# Get the data, ROIs, the flip angle pattern and the colormaps
data_folder = Path(tempfile.mkdtemp())
dataset = '14251660'
zenodo_get.zenodo_get([dataset, '-r', 5, '-o', data_folder])  # r: retries
with zipfile.ZipFile(data_folder / Path('open_source_cmrf_scanner_comparison.zip'), 'r') as zip_ref:
    zip_ref.extractall(data_folder)

### 3) Define image reconstruction and parameter estimation methods for cMRF and reference sequences

For all scans we carry out dictionary matching to estimate the quantitative parameters from a series of qualitative images. So let's start by defining a function for this.

In [ ]:
#Function to calculate the dictionary matching
def dictionary_matching(img_data, model, dictionary_values):
    dictionary_values = dictionary_values.to(dtype=torch.float32)
    (signal_dictionary,) = model(torch.ones(1), dictionary_values)
    signal_dictionary = signal_dictionary.to(dtype=torch.complex64)
    vector_norm = torch.linalg.vector_norm(signal_dictionary, dim=0)
    signal_dictionary /= vector_norm
    signal_dictionary = signal_dictionary.to(img_data.dtype)

    # Calculate the dot-product 
    # and select for each voxel the values that correspond to the maximum of the dot-product
    n_y, n_x = img_data.shape[-2:]
    dot_product = torch.mm(rearrange(img_data, 'other 1 z y x->(z y x) other'), signal_dictionary)
    idx_best_match = torch.argmax(torch.abs(dot_product), dim=1)
    return rearrange(dictionary_values[idx_best_match], '(y x)->1 1 y x', y=n_y, x=n_x)
          

Now we define the reconstruction and parameter estimation for the reference sequences. We use a DirectReconstruction which in this case carries out a Fast Fourier Transform. The data from the different receiver coils were combined by a weighted sum using coil sensitivity maps.

In [ ]:
#Function to reconstruct the reference scans
def multi_image_reco(kdata):
    reco = DirectReconstruction(kdata=kdata, csm=None)
    img = reco(kdata)
    first_image = IData(data=img.data[0,None], header=img.header)
    csm_first_image = CsmData.from_idata_inati(first_image)
    reco = DirectReconstruction(kdata=kdata, csm=csm_first_image)
    return reco(kdata)

# Dictionary matching using a mono-exponential model for T2
def dictionary_matching_spin_echo_t2(img, t2):
    model = MonoExponentialDecay(decay_time=img.header.te*1000)
    return dictionary_matching(img.data, model, dictionary_values=t2)
    
# Dictionary matching using a inversion recovery model for T1    
def dictionary_matching_spin_echo_t1(img, t1):
    model = MagnitudeOp() @ InversionRecovery(ti=img.header.ti*1000)
    return dictionary_matching(img.data.abs(), model, dictionary_values=t1)

# Reconstruct T1 and T2 spin-echo reference scans and return T1 and T2 maps
def reco_ref_scans(pname_ref_t1, fname_t1_se_pulseq, pname_ref_t2, fname_t2_se_pulseq, t1, t2):
    # Multi echo SE pulseq
    kdata = KData.from_file( pname_ref_t2 / fname_t2_se_pulseq, KTrajectoryCartesian())
    kdata.header.recon_matrix.y = 128
    img_multi_echo_se_pulseq = multi_image_reco(kdata)
    t2_map_se_pulseq = dictionary_matching_spin_echo_t2(img_multi_echo_se_pulseq, t2)[0,0,...]
        
    # Multi TI SE pulseq
    kdata = KData.from_file(pname_ref_t1 / fname_t1_se_pulseq, KTrajectoryCartesian())
    kdata.header.recon_matrix.y = 128
    kdata.header.ti = torch.as_tensor([25, 50, 300, 600, 1200, 2400, 4800])/1000
    img_multi_ti_se_pulseq = multi_image_reco(kdata)
    t1_map_se_pulseq = dictionary_matching_spin_echo_t1(img_multi_ti_se_pulseq, t1)[0,0,...]
    
    return t1_map_se_pulseq, t2_map_se_pulseq

Finally we define the function to reconstruct the cMRF data and estimate the $T_1$ and $T_2$ maps.

In [ ]:
#Function to reconstruct the cMRF scans
def reco_cMRF_scans(pname, scan_name, fa, t1, t2):
    n_lines_per_img = 20
    n_lines_overlap= 10

    # Image reconstruction of average image
    kdata = KData.from_file(pname / scan_name, KTrajectoryIsmrmrd())
    avg_recon = DirectReconstruction(kdata)
    
    # Split data into dynamics and reconstruct
    dyn_idx = split_idx(torch.arange(0,47), n_lines_per_img, n_lines_overlap)
    dyn_idx = torch.cat([dyn_idx + ind*47 for ind in range(15)], dim=0)
    
    kdata_dyn = kdata.split_k1_into_other(dyn_idx, other_label='repetition')
    
    dyn_recon = DirectReconstruction(kdata_dyn, csm=avg_recon.csm)
    dcf_data_dyn = rearrange(avg_recon.dcf.data, 'k2 k1 other k0->other k2 k1 k0')
    dcf_data_dyn = rearrange(dcf_data_dyn[dyn_idx.flatten(),...], '(other k1) 1 k2 k0->other k2 k1 k0', k1=dyn_idx.shape[-1])
    dyn_recon.dcf = DcfData(dcf_data_dyn)

    img = dyn_recon(kdata_dyn).rss()[:,0,:,:]

    # Dictionary settings
    t1, t2 = torch.broadcast_tensors(t1[None,:], t2[:,None])
    t1_all = t1.flatten().to(dtype=torch.float32)
    t2_all = t2.flatten().to(dtype=torch.float32)
    
    t1 = t1_all[t1_all >= t2_all]
    t2 = t2_all[t1_all >= t2_all]
    m0 = torch.ones_like(t1)

    # Dictionary calculation
    n_rf_pulses_per_block = 47 # 47 RF pulses in each block
    acq_t_ms = kdata.header.acq_info.acquisition_time_stamp[0,0,:,0]*2.5
    delay_between_blocks = [acq_t_ms[n_block*n_rf_pulses_per_block] - acq_t_ms[n_block*n_rf_pulses_per_block-1] for n_block in range(1,3*5)]
    delay_between_blocks.append(delay_between_blocks[-1]) # last delay is not needed but makes computations easier

    flip_angles = fa
    rf_phases = 0.0
    te = 1.52
    tr = kdata.header.tr * 1000
    inv_prep_ti = [21,None,None,None,None]*3 # 21 ms delay after inversion pulse in block 0
    t2_prep_te = [None,None,30,50,100]*3 # T2-preparation pulse with TE = 30, 50, 100
    delay_due_to_prep = [0, 30, 50, 100, 21]*3
    delay_after_block = [trig_delay-prep_delay for prep_delay, trig_delay in zip(delay_due_to_prep, delay_between_blocks)]
    epg_mrf_fisp = EpgMrfFispWithPreparation(flip_angles, rf_phases, te, tr, inv_prep_ti, t2_prep_te, n_rf_pulses_per_block, delay_after_block)
    (signal_dictionary,) = epg_mrf_fisp.forward(m0, t1, t2)


    signal_dictionary = rearrange(signal_dictionary[dyn_idx.flatten(),...], '(other k1) t->other t k1', k1=dyn_idx.shape[-1])
    signal_dictionary = torch.mean(signal_dictionary, dim=-1)
    signal_dictionary = signal_dictionary.abs()

    # Normalise dictionary entries
    vector_norm = torch.linalg.vector_norm(signal_dictionary, dim=0)
    signal_dictionary /= vector_norm

    # Dictionary matching
    n_y, n_x = img.shape[-2:]
    dot_product = torch.mm(rearrange(img.abs(), 'other y x->(y x) other'), signal_dictionary)
    idx_best_match = torch.argmax(torch.abs(dot_product), dim=1)
    t1_match = rearrange(t1[idx_best_match], '(y x)->y x', y=n_y, x=n_x)
    t2_match = rearrange(t2[idx_best_match], '(y x)->y x', y=n_y, x=n_x)
    
    return t1_match, t2_match

### 4) Define evaluation methods

The phantom we used for data acquisition consists of nine tubes, each representing a different cardiac tissue type. Now we define a method to calculate the mean value and standard deviation over a circular ROI in each of these tubes.

In [ ]:
#Function to calculate the mean and standard deviation of the tubes in the image data
def image_statistics(idat, mask_name):
    
    if mask_name is not None:
        mask = np.squeeze(np.load(mask_name))
        
    number_of_tubes = 9
    mean = []
    std_deviation = []
    for idx_value in range(number_of_tubes):                      
        mean.append(torch.mean(idat[mask==idx_value+1]))
        std_deviation.append(torch.std(idat[mask==idx_value+1]))

    return mean, std_deviation

### 5) Run through all datasets and calculate $T_1$ and $T_2$ maps

Now we can go through the acquisitions at the different scanners, reconstruct the cMRF and reference scans, estimate $T_1$ and $T_2$ maps.

In [ ]:
# Define the T1 and T2 values to be included in the dictionaries
t1 = torch.cat((torch.arange(50, 2000+10, 10), torch.arange(2020, 3000+20, 20), torch.arange(3050,5000+50,50)))
t2 = torch.cat((torch.arange(6, 100+2, 2), torch.arange(105, 200+5, 5), torch.arange(220,500+20,20)))
    
# Read in flip angle pattern
fname_angle = data_folder / Path('cMRF_fa_705rep.txt')

with open(fname_angle, "r") as file:
    fa = torch.as_tensor([float(line) for line in file.readlines()])/180 * torch.pi

cmrf_t1_maps = []
cmrf_t2_maps = []
ref_t1_maps = []
ref_t2_maps = []

for scanner_idx in range(4):
    # Current path of data
    pname = data_folder / Path(f'scanner{scanner_idx+1}/')
    
    # Reference T1 and T2 maps
    t1_map_ref, t2_map_ref =  reco_ref_scans(pname, 'ref_t1.h5', pname, 'ref_t2.h5', t1, t2)  
    ref_t1_maps.append(t1_map_ref)
    ref_t2_maps.append(t2_map_ref)
    
    # cMRF T1 and T2 maps
    t1_map_cmrf, t2_map_cmrf = reco_cMRF_scans(pname, 'cMRF.h5', fa, t1, t2)
    cmrf_t1_maps.append(t1_map_cmrf)
    cmrf_t2_maps.append(t2_map_cmrf)

    


### 6) Visualise and evaluate results

Now we visualise and compare all the results.

In [ ]:
# Create recommended colormaps
cmap_t1 = ListedColormap(np.loadtxt(data_folder / Path('lipari.csv')))
cmap_t2 = ListedColormap(np.loadtxt(data_folder / Path('navia.csv')))

# Plot T1 and T2 maps
for scanner_idx in range(4):
    fig, ax = plt.subplots(2,2)
    for cax in ax.flatten():
        cax.set_axis_off()

    im = ax[0,0].imshow(cmrf_t1_maps[scanner_idx], vmin=0, vmax=2000, cmap=cmap_t1)
    ax[0,0].set_title('cMRF T1 (ms)')
    plt.colorbar(im)
    im = ax[1,0].imshow(cmrf_t2_maps[scanner_idx], vmin=0, vmax=200, cmap=cmap_t2)
    ax[1,0].set_title('cMRF T2 (ms)')
    plt.colorbar(im)
    
    ax[0,1].imshow(ref_t1_maps[scanner_idx], vmin=0, vmax=2000, cmap=cmap_t1)
    ax[0,1].set_title('Reference T1 (ms)')
    ax[1,1].imshow(ref_t2_maps[scanner_idx], vmin=0, vmax=200, cmap=cmap_t2)
    ax[1,1].set_title('Reference T1 (ms)')

    
    plt.tight_layout()    
    plt.show()

In [ ]:
# Compare ROI values
fig, ax = plt.subplots(2,4, figsize=(12,7))
for scanner_idx in range(4):
    # Current path of data
    pname = data_folder / Path(f'scanner{scanner_idx+1}/')
    
    t1_mean_cmrf, t1_std_cmrf = image_statistics(cmrf_t1_maps[scanner_idx], pname / "mask.npy") 
    t2_mean_cmrf, t2_std_cmrf = image_statistics(cmrf_t2_maps[scanner_idx], pname / "mask.npy") 
    t1_mean_ref, t1_std_ref = image_statistics(ref_t1_maps[scanner_idx], pname / "mask.npy") 
    t2_mean_ref, t2_std_ref = image_statistics(ref_t2_maps[scanner_idx], pname / "mask.npy") 
    
    ax[0,scanner_idx].set_title(f'Scanner {scanner_idx+1}')
    
    # Plot T1 data
    ax[0,scanner_idx].errorbar(t1_mean_ref, t1_mean_cmrf, t1_std_cmrf, t1_std_ref, fmt="o", color="teal")
    ax[0,scanner_idx].plot([0, 2000], [0, 2000], color="darkorange")
    
    # Plot T2 data
    ax[1,scanner_idx].errorbar(t2_mean_ref, t2_mean_cmrf, t2_std_cmrf, t2_std_ref, fmt="o", color="teal")
    ax[1,scanner_idx].plot([0, 200], [0, 200], color="darkorange")
    
    for pidx in range(2):
        ax[pidx,scanner_idx].set_xlabel(f'T{int(pidx+1)} - Reference (ms)',fontsize=12)
        ax[pidx,scanner_idx].set_ylabel(f'T{int(pidx+1)} - cMRF (ms)',fontsize=12)
        ax[pidx,scanner_idx].grid()
        ax[pidx,scanner_idx].set_aspect('equal', adjustable='box')
    
    
    plt.tight_layout()    